In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

# 1. Display all the header tags from wikipedia.org

In [2]:
def scrape(url):
    page=requests.get(url)
    soup=BeautifulSoup(page.content)
    header = soup.find_all(['h1', 'h2','h3','h4','h5','h6'])
    print('List of all the header tags :', *header, sep='\n\n')

In [3]:
scrape("https://www.wikipedia.org")

List of all the header tags :

<h1 class="central-textlogo-wrapper">
<span class="central-textlogo__image sprite svg-Wikipedia_wordmark">
Wikipedia
</span>
<strong class="jsl10n localized-slogan" data-jsl10n="portal.slogan">The Free Encyclopedia</strong>
</h1>

<h2 class="bookshelf-container">
<span class="bookshelf">
<span class="text">
<bdi dir="ltr">
1 000 000+
</bdi>
<span class="jsl10n" data-jsl10n="entries">
articles
</span>
</span>
</span>
</h2>

<h2 class="bookshelf-container">
<span class="bookshelf">
<span class="text">
<bdi dir="ltr">
100 000+
</bdi>
<span class="jsl10n" data-jsl10n="portal.entries">
articles
</span>
</span>
</span>
</h2>

<h2 class="bookshelf-container">
<span class="bookshelf">
<span class="text">
<bdi dir="ltr">
10 000+
</bdi>
<span class="jsl10n" data-jsl10n="portal.entries">
articles
</span>
</span>
</span>
</h2>

<h2 class="bookshelf-container">
<span class="bookshelf">
<span class="text">
<bdi dir="ltr">
1 000+
</bdi>
<span class="jsl10n" data-jsl10n=

# 2. Display IMDB’s Top rated 100 movies’ data (i.e. name, rating, year of release)

In [4]:
def scrap_movie(url):
    movies=[]
    year = []
    ratings=[]
    data=pd.DataFrame()
    
    page=requests.get(url)
    soup=BeautifulSoup(page.content)
    
    # scrapping movie name
    scrapped_movies = soup.find_all('td',class_='titleColumn')
    
    for movie in scrapped_movies:
        movie=movie.get_text().replace('\n',"")
        movie=movie.strip("")
        movies.append(movie)
    
    for i in movies:
        y=re.findall('\d+', i )
        y=y[-1]
        year.append(y)
    
    # extracting movie name from scrapped data
    scrapped_ratings = soup.find_all('td',class_='ratingColumn imdbRating')
    
    for rating in scrapped_ratings:
        rating=rating.get_text().replace('\n',"")
        rating=rating.strip("")
        ratings.append(rating)
    
    # creating dataframe
    data['Movie Name']=movies[:100]
    data['Year']=year[:100]
    data['Ratings']=ratings[:100]

    return data

In [5]:
df=scrap_movie("https://www.imdb.com/chart/top")
df.head()

,Movie Name,Year,Ratings
0,1. The Shawshank Redemption(1994),1994,9.2
1,2. The Godfather(1972),1972,9.1
2,3. The Godfather: Part II(1974),1974,9.0
3,4. The Dark Knight(2008),2008,9.0
4,5. 12 Angry Men(1957),1957,8.9


# 3. Display IMDB’s Top rated 100 Indian movies’ data

In [6]:
df=scrap_movie("https://www.imdb.com/india/top-rated-indian-movies/")
df.head()

,Movie Name,Year,Ratings
0,1. Jai Bhim(2021),2021,8.6
1,2. Nayakan(1987),1987,8.5
2,3. Pariyerum Perumal(2018),2018,8.5
3,4. Anbe Sivam(2003),2003,8.5
4,5. C/o Kancharapalem(2018),2018,8.5


# 4.a. Top 10 ODI teams in men’s cricket 

In [7]:
def scrap_team(url):
    teams=[]
    match = []
    ratings=[]
    data=pd.DataFrame()
    
    page=requests.get(url)
    soup=BeautifulSoup(page.content)

    # scrapping team details at position 1
    
    scrapped_team = soup.find('td',class_='rankings-block__banner--team-name')
    team=scrapped_team.get_text().split('\n')
    team=team[2]
    teams.append(team)
    
    scrapped_match = soup.find('td',class_='rankings-block__banner--matches')
    mat=scrapped_match.get_text().split('\n')  
    
    scrapped_point = soup.find('td',class_='rankings-block__banner--points')
    point=scrapped_point.get_text().split('\n')    
    
    scrapped_rating = soup.find('td',class_='rankings-block__banner--rating u-text-right')
    rating=scrapped_rating.get_text().split()
    ratings.append(rating)     
    
    # scrapping teams details at subsequent positions 
    
    scrapped_team = soup.find_all('td',class_='table-body__cell rankings-table__team')
    for team in scrapped_team:
        team=team.get_text().split('\n')
        team=team[2]
        teams.append(team)
        
    scrapped_m = soup.find_all('td',class_='table-body__cell u-center-text')
    for m in scrapped_m:
        m=m.get_text().split()
        match.append(m)
        
    matches=match[::2]
    matches.insert(0,mat)
    points=match[1::2]
    points.insert(0,point)
  
    scrapped_rating = soup.find_all('td',class_='table-body__cell u-text-right rating')
    for rating in scrapped_rating:
        rating=rating.get_text().split()
        ratings.append(rating)
    
    # creating dataframe
    data['Teams']=teams[:10]
    data['Matches']=matches[:10]
    data['Points']=points[:10]
    data['Ratings']=ratings[:10]

    return data

In [8]:
df=scrap_team("https://www.icc-cricket.com/rankings/mens/team-rankings/odi")
df

,Teams,Matches,Points,Ratings
0,New Zealand,[17],"[2,054]",[121]
1,England,[32],"[3,793]",[119]
2,Australia,[28],"[3,244]",[116]
3,India,[32],"[3,624]",[113]
4,South Africa,[25],"[2,459]",[98]
5,Pakistan,[27],"[2,524]",[93]
6,Bangladesh,[30],"[2,740]",[91]
7,West Indies,[30],"[2,523]",[84]
8,Sri Lanka,[32],"[2,657]",[83]
9,Afghanistan,[17],"[1,054]",[62]


# 4.b. Top 10 ODI Batsmen in men

In [9]:
def scrap_batsmen(url):
    players=[]
    countries = []
    ratings=[]
    data=pd.DataFrame()
    
    page=requests.get(url)
    soup=BeautifulSoup(page.content)

    # scrapping player name at position 1
    
    player=soup.find('div',class_='rankings-block__banner--name')
    player=player.get_text()
    players.append(player)
    
    a=soup.find('div',class_='rankings-block__banner--nationality')
    a=a.get_text().split()
    countries.append(a[0])
    ratings.append(a[1])
        
    
    # scrapping players at subsequent position 
    
    scrapped_player = soup.find_all('td',class_='table-body__cell name')
    for player in scrapped_player:
        player=player.get_text().split('\n')
        player=player[1]
        players.append(player)
        
    scrapped_country = soup.find_all('td',class_='table-body__cell nationality-logo')
    for country in scrapped_country:
        country=country.get_text().split()
        countries.append(country)
         
    scrapped_rating = soup.find_all('td',class_='table-body__cell u-text-right rating')
    for rating in scrapped_rating:
        rating=rating.get_text().split()
        ratings.append(rating)
    
    # creating dataframe
    data['Player_Name']=players[:10]
    data['Country']=countries[:10]
    data['Ratings']=ratings[:10]

    return data

In [10]:
df=scrap_batsmen("https://www.icc-cricket.com/rankings/mens/player-rankings/odi")
df

,Player_Name,Country,Ratings
0,Babar Azam,PAK,873
1,Virat Kohli,[IND],[844]
2,Rohit Sharma,[IND],[813]
3,Ross Taylor,[NZ],[801]
4,Aaron Finch,[AUS],[779]
5,Jonny Bairstow,[ENG],[775]
6,David Warner,[AUS],[762]
7,Shai Hope,[WI],[758]
8,Kane Williamson,[NZ],[754]
9,Quinton de Kock,[SA],[743]


# 4.c. Top 10 ODI bowlers in men

In [11]:
def scrap_bowler(url):
    players=[]
    countries = []
    ratings=[]
    data=pd.DataFrame()
    
    page=requests.get(url)
    soup=BeautifulSoup(page.content)

    # # scrapping player name at position 1
    a=[]
    scrapped_player = soup.find_all('div',class_='rankings-block__banner--name')
    for player in scrapped_player:
        player=player.get_text().split('\n')
        a.append(player)
        
    a1=a[1]
    
    b=[]
    a = soup.find_all('div',class_='rankings-block__banner--nationality')
    for i in a:
        i=i.get_text().split('\n')
        b.append(i)
    b1=b[1][2]
    c1=b[1][3]
    
        
    
    # scrapping players at subsequent position 
    
    scrapped_player = soup.find_all('td',class_='table-body__cell name')
    for player in scrapped_player:
        player=player.get_text().split('\n')
        player=player[1]
        players.append(player)
        
    scrapped_country = soup.find_all('td',class_='table-body__cell nationality-logo')
    for country in scrapped_country:
        country=country.get_text().split()
        countries.append(country)
         
    scrapped_rating = soup.find_all('td',class_='table-body__cell u-text-right rating')
    for rating in scrapped_rating:
        rating=rating.get_text().split()
        ratings.append(rating)
    
    # creating dataframe
    players=players[9:18]
    players.insert(0,a1)
    countries=countries[9:18]
    countries.insert(0,b1)
    ratings=ratings[9:18]
    ratings.insert(0,c1)
    data['Player_Name']=players
    data['Country']=countries
    data['Ratings']=ratings

    return data

In [12]:
df=scrap_bowler("https://www.icc-cricket.com/rankings/mens/player-rankings/odi")
df

,Player_Name,Country,Ratings
0,[Trent Boult],NZ,737
1,Josh Hazlewood,[AUS],[709]
2,Mujeeb Ur Rahman,[AFG],[708]
3,Chris Woakes,[ENG],[700]
4,Mehedi Hasan,[BAN],[692]
5,Matt Henry,[NZ],[691]
6,Jasprit Bumrah,[IND],[679]
7,Mitchell Starc,[AUS],[652]
8,Shakib Al Hasan,[BAN],[650]
9,Kagiso Rabada,[SA],[643]


# 5.a. Top 10 ODI teams in women’s cricket 

In [13]:
df=scrap_team("https://www.icc-cricket.com/rankings/womens/team-rankings/odi")
df

,Teams,Matches,Points,Ratings
0,Australia,[17],"[2,746]",[162]
1,South Africa,[19],"[2,307]",[121]
2,England,[18],"[2,148]",[119]
3,India,[17],"[1,899]",[112]
4,Bangladesh,[5],[475],[95]
5,New Zealand,[19],"[1,668]",[88]
6,West Indies,[19],"[1,658]",[87]
7,Pakistan,[18],"[1,226]",[68]
8,Ireland,[5],[240],[48]
9,Sri Lanka,[5],[233],[47]


# 5.b Top 10 women’s ODI players

In [14]:
df=scrap_batsmen("https://www.icc-cricket.com/rankings/womens/player-rankings/odi")
print("Top 10 ODI Batsmen in women : \n","-"*32)
df

Top 10 ODI Batsmen in women : 
 --------------------------------


,Player_Name,Country,Ratings
0,Lizelle Lee,SA,761
1,Alyssa Healy,[AUS],[750]
2,Mithali Raj,[IND],[738]
3,Tammy Beaumont,[ENG],[728]
4,Amy Satterthwaite,[NZ],[717]
5,Smriti Mandhana,[IND],[710]
6,Meg Lanning,[AUS],[699]
7,Beth Mooney,[AUS],[690]
8,Stafanie Taylor,[WI],[676]
9,Heather Knight,[ENG],[674]


In [15]:
df=scrap_bowler("https://www.icc-cricket.com/rankings/womens/player-rankings/odi")
print("Top 10 ODI Bowler in women : \n","-"*32)
df

Top 10 ODI Bowler in women : 
 --------------------------------


,Player_Name,Country,Ratings
0,[Jess Jonassen],AUS,760
1,Jhulan Goswami,[IND],[727]
2,Megan Schutt,[AUS],[717]
3,Marizanne Kapp,[SA],[715]
4,Sophie Ecclestone,[ENG],[701]
5,Shabnim Ismail,[SA],[688]
6,Katherine Brunt,[ENG],[666]
7,Ayabonga Khaka,[SA],[643]
8,Anya Shrubsole,[ENG],[598]
9,Kate Cross,[ENG],[589]


# 5.c. Top 10 women’s ODI all-rounder

In [16]:
def scrap_allrounder(url):
    players=[]
    countries = []
    ratings=[]
    data=pd.DataFrame()
    
    page=requests.get(url)
    soup=BeautifulSoup(page.content)

    # scrapping player name at position 1
    a=[]
    scrapped_player = soup.find_all('div',class_='rankings-block__banner--name')
    for player in scrapped_player:
        player=player.get_text().split('\n')
        a.append(player)
        
    a1=a[2]
    
    b=[]
    a = soup.find_all('div',class_='rankings-block__banner--nationality')
    for i in a:
        i=i.get_text().split('\n')
        b.append(i)
    b1=b[2][2]
    c1=b[2][3]
    
        
    
    # scrapping players at subsequent position 
    
    scrapped_player = soup.find_all('td',class_='table-body__cell name')
    for player in scrapped_player:
        player=player.get_text().split('\n')
        player=player[1]
        players.append(player)
        
    scrapped_country = soup.find_all('td',class_='table-body__cell nationality-logo')
    for country in scrapped_country:
        country=country.get_text().split()
        countries.append(country)
         
    scrapped_rating = soup.find_all('td',class_='table-body__cell u-text-right rating')
    for rating in scrapped_rating:
        rating=rating.get_text().split()
        ratings.append(rating)
    
    # creating dataframe
    players=players[18:27]
    players.insert(0,a1)
    countries=countries[18:27]
    countries.insert(0,b1)
    ratings=ratings[18:27]
    ratings.insert(0,c1)
    data['Player_Name']=players
    data['Country']=countries
    data['Ratings']=ratings

    return data

In [17]:
df=scrap_allrounder("https://www.icc-cricket.com/rankings/womens/player-rankings/odi")
df

,Player_Name,Country,Ratings
0,[Marizanne Kapp],SA,384
1,Natalie Sciver,[ENG],[372]
2,Ellyse Perry,[AUS],[365]
3,Stafanie Taylor,[WI],[319]
4,Deepti Sharma,[IND],[299]
5,Ashleigh Gardner,[AUS],[275]
6,Dane van Niekerk,[SA],[274]
7,Hayley Matthews,[WI],[272]
8,Jess Jonassen,[AUS],[272]
9,Katherine Brunt,[ENG],[272]


# 8. scrape details from dineout.co.in

In [22]:
def scrape_dining(url):
    names=[]
    cuisines = []
    locations=[]
    ratings=[]
    links=[]
    data=pd.DataFrame()
    
    page=requests.get(url)
    soup=BeautifulSoup(page.content)
    
    # scrapping names
    scrapped_name = soup.find_all('a',class_='restnt-name ellipsis')
    for name in scrapped_name:
        name=name.get_text()
        names.append(name)
        
    # scrapping links
    l=[]
    scrapped_link = soup.find_all('img',class_='lazy-load-img no-img')
    for link in scrapped_link:
        l.append(link)
        
    for i in range(len(l)):
        links.append(l[i].attrs['data-src'])

    # scrapping ratings
    scrapped_rating = soup.find_all('div',class_='restnt-rating rating-4')
    for rating in scrapped_rating:
        rating=rating.get_text()
        ratings.append(rating)

    # scrapping cuisines
    scrapped_cuisine = soup.find_all('span',class_='double-line-ellipsis')
    for cuisine in scrapped_cuisine:
        cuisine=cuisine.get_text().split("|")
        cuisines.append(cuisine[1])
        
    # scrapping location
    scrapped_location = soup.find_all('div',class_='restnt-loc ellipsis')
    for location in scrapped_location:
        location=location.get_text()
        locations.append(location)
    
    # creating dataframe
    data['Name']=names
    data['Locations']=locations
    data['Cuisine']=cuisines
    data['Rating']=ratings
    data['URL']=links

    return data

In [23]:
df=scrape_dining("https://www.dineout.co.in/delhi-restaurants/buffet-special")
df

,Name,Locations,Cuisine,Rating,URL
0,Castle Barbeque,"Connaught Place, Central Delhi","Chinese, North Indian",3.5,https://im1.dineout.co.in/images/uploads/resta...
1,Jungle Jamboree,"3CS Mall,Lajpat Nagar - 3, South Delhi","North Indian, Barbecue, Italian, Asian",3.9,https://im1.dineout.co.in/images/uploads/resta...
2,Castle Barbeque,"Pacific Mall,Tagore Garden, West Delhi","North Indian, Chinese",4,https://im1.dineout.co.in/images/uploads/resta...
3,Cafe Knosh,"The Leela Ambience Convention Hotel,Shahdara, ...","Multi-Cuisine, North Indian, Italian, Contine...",4.3,https://im1.dineout.co.in/images/uploads/resta...
4,The Barbeque Company,"Gardens Galleria,Sector 38A, Noida","Barbecue, Chinese, Mughlai, North Indian",4.1,https://im1.dineout.co.in/images/uploads/resta...
5,India Grill,"Hilton Garden Inn,Saket, South Delhi","North Indian, Italian, Oriental",3.9,https://im1.dineout.co.in/images/uploads/resta...
6,Delhi Barbeque,"Taurus Sarovar Portico,Mahipalpur, South Delhi","Barbecue, North Indian",3.6,https://im1.dineout.co.in/images/uploads/resta...
7,The Monarch - Bar Be Que Village,"Indirapuram Habitat Centre,Indirapuram, Ghaziabad","North Indian, Chinese, Fast Food",3.9,https://im1.dineout.co.in/images/uploads/resta...
8,World Cafe,"Vibe by The Lalit Traveller,Sector 35, Faridabad","North Indian, Chinese, Continental",4.2,https://im1.dineout.co.in/images/uploads/resta...
9,Indian Grill Room,"Suncity Business Tower,Golf Course Road, Gurgaon","North Indian, Mughlai, Barbecue",4.3,https://im1.dineout.co.in/images/uploads/resta...
